In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
fake_df=pd.read_csv("Fake.csv")
real_df=pd.read_csv("True.csv")

In [3]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
real_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake_df['label']=0
real_df['label']=1

In [6]:
df=pd.concat([fake_df,real_df],ignore_index=True)

In [7]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [8]:
df.shape

(44898, 5)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [10]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [12]:
def preprocess_text(text):
    text=text.lower()

    text=text.translate(str.maketrans('', '',string.punctuation))

    tokens=text.split()
    stop_words=set(stopwords.words('english'))
    tokens=[word for word in tokens if word not in stop_words]

    lemmatizer=WordNetLemmatizer()
    tokens=[lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

    

In [13]:
df['clean_text']=df['text'].apply(preprocess_text)

df[['text','clean_text']].head()

,text,clean_text
0,Donald Trump just couldn t wish all Americans ...,donald trump wish american happy new year leav...
1,House Intelligence Committee Chairman Devin Nu...,house intelligence committee chairman devin nu...
2,"On Friday, it was revealed that former Milwauk...",friday revealed former milwaukee sheriff david...
3,"On Christmas day, Donald Trump announced that ...",christmas day donald trump announced would bac...
4,Pope Francis used his annual Christmas Day mes...,pope francis used annual christmas day message...


In [14]:
from sklearn.model_selection import train_test_split

X=df['clean_text']
y=df['label']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print("Training set size:",X_train.shape)
print("Test set size:",X_test.shape)

Training set size: (35918,)
Test set size: (8980,)


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(max_features=5000)

X_train_tfidf=tfidf.fit_transform(X_train)
X_test_tfidf=tfidf.transform(X_test)

print("TF-IDF training shape:",X_train_tfidf.shape)
print("TF-IDF testing shape:",X_test_tfidf.shape)

TF-IDF training shape: (35918, 5000)
TF-IDF testing shape: (8980, 5000)


In [17]:
from sklearn.linear_model import LogisticRegression

model=LogisticRegression()

model.fit(X_train_tfidf,y_train)

LogisticRegression()

In [19]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

y_pred=model.predict(X_test_tfidf)

print("Accuracy:",accuracy_score(y_test,y_pred))

print("\n Classification report:",classification_report(y_test,y_pred))

print("\n Confusion Matrix:",confusion_matrix(y_test,y_pred))

Accuracy: 0.9897550111358575

 Classification report:               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4733
           1       0.99      0.99      0.99      4247

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980


 Confusion Matrix: [[4676   57]
 [  35 4212]]


In [22]:
import pickle

with open('fake_news_model.pkl','wb') as model_file:
    pickle.dump(model,model_file)

with open('tfidf_vectorizer.pkl','wb') as vec_file:
    pickle.dump(tfidf,vec_file)

print("Model and vectorizer saved successfully")

Model and vectorizer saved successfully


In [23]:
with open('fake_news_model.pkl','rb') as model_file:
    loaded_model=pickle.load(model_file)

with open('tfidf_vectorizer.pkl','rb') as vec_file:
    loaded_vectorizer=pickle.load(vec_file)

sample_text=["Breaking News: Government launches new policy to combat climate change"]

cleaned=preprocess_text(sample_text[0])
vectorized_input=loaded_vectorizer.transform([cleaned])

prediction=loaded_model.predict(vectorized_input)
print("Prediction:","Real" if prediction[0]==1 else "Fake")

Prediction: Fake
